In [1]:
"""
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'
}

configuration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))
"""

"\nimport ibmos2spark\n# @hidden_cell\ncredentials = {\n    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',\n    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',\n    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',\n    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'\n}\n\nconfiguration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'\ncos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')\n\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.getOrCreate()\n\ndf = spark.read  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')  .option('header', 'true')  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))\n"

In [2]:
"""
    .config("spark.memory.fraction","0.6") \
    .config("spark.memory.storageFraction","0.5")\
    .config("spark.memory.offHeap.enabled",True) \
    .config("spark.memory.offHeap.size","4g")  \
    .config("spark.executor.heartbeatInterval", "10000s")  \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.executor.extraJavaOptions"," \
                -XX:+UseG1GC \
                -XX:+PrintFlagsFinal \
                -XX:+PrintReferenceGC \
                -verbose:gc \
                -XX:+PrintGCDetails \
                -XX:+PrintGCTimeStamps \
                -XX:+PrintAdaptiveSizePolicy \
                -XX:+UnlockDiagnosticVMOptions \
                -XX:+G1SummarizeConcMark \
                -Xms13g \
                -Xmx13g \
                -XX:InitiatingHeapOccupancyPercent=35 \
                -XX:ConcGCThread=20 \
                ") \
"""

'\n    .config("spark.memory.fraction","0.6")     .config("spark.memory.storageFraction","0.5")    .config("spark.memory.offHeap.enabled",True)     .config("spark.memory.offHeap.size","4g")      .config("spark.executor.heartbeatInterval", "10000s")      .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")     .config("spark.executor.extraJavaOptions","                 -XX:+UseG1GC                 -XX:+PrintFlagsFinal                 -XX:+PrintReferenceGC                 -verbose:gc                 -XX:+PrintGCDetails                 -XX:+PrintGCTimeStamps                 -XX:+PrintAdaptiveSizePolicy                 -XX:+UnlockDiagnosticVMOptions                 -XX:+G1SummarizeConcMark                 -Xms13g                 -Xmx13g                 -XX:InitiatingHeapOccupancyPercent=35                 -XX:ConcGCThread=20                 ") '

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

try:
    spark.close()
except: 
    pass

try:
    conf = SparkConf().setAppName('Spark Project')
    conf.set('executor-memory','6')
    conf.set('spark.driver.memory', '8g')
    conf.set('spark.executor.cores', '4')
    conf.set("spark.num.executors","6")
    conf.set('spark.memory.fraction','.6')
    conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

    sc = SparkContext(conf=conf)

except Exception as e:
    print(e)
#.config("spark.executor.cores", "4") \
#.config("spark.default.parallelism", "6") \ # No more than availible Cores
#.config("spark.executor.memoryOverhead",200) \
# spark.num.executors is set since we dont have enough ram for 8 
# Note: 6 running processes each having 2gb => 12 gb ram used
spark = SparkSession.builder \
    .appName("Spark Project") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

df = spark.read.load("data/US_Accidents_Dec19.csv",format="csv", sep=",", inferSchema="true", header="true")

In [4]:
sc

<SparkContext master=local[*] appName=Spark Project>

In [5]:
sc._conf.getAll()

[('spark.executor.id', 'driver'),
 ('spark.driver.host', 'localhost'),
 ('spark.app.id', 'local-1588449089354'),
 ('spark.app.name', 'Spark Project'),
 ('spark.executor.cores', '4'),
 ('executor-memory', '6'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.num.executors', '6'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.memory.fraction', '.6'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '46099')]

# Imports

In [6]:
from pyspark.sql.functions import when, count, col, isnan, countDistinct,from_unixtime,from_utc_timestamp, unix_timestamp,split, to_timestamp, hour, month, lit,collect_list
import seaborn as sns 
import matplotlib.pyplot as plt 

# 1. Define variables

In [7]:
colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

# Dropping data that cant help during model
df = df.drop(*colRem)

# Convert boolean to string since PCA cant handle boolean which should be a class
df = df.select(*[col(c[0]).cast("string").alias(c[0]) if c[1] == 'boolean' else col(c[0]).alias(c[0]) for c in df.dtypes])

#renamedHousing.select([count(when(col(c).isNull(), c)).alias(c) for c in colNum]).show()
rdd = sc.parallelize(df.dtypes)

colCat = rdd.map(lambda i: i[0] if (i[1]=='string' or i[1]=='boolean' and i[0]) else None).filter(lambda i: i != None).collect()
colNum = rdd.map(lambda i: i[0] if (i[1]=='double' and i[0]) else None).filter(lambda i: i != None).collect()

print(colCat)
print(colNum)

['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


# 2. Preprocessing

In [8]:
from pyspark.ml.feature import VectorAssembler,OneHotEncoderEstimator,QuantileDiscretizer, StringIndexer, Imputer,StandardScaler,MinMaxScaler
from pyspark.ml import Pipeline
from numpy import argmax

## Analyse single city
This could be CA since it stands for a little less than 50% of the total set

In [9]:
#df = df.filter(df.State == "CA") # Lowers the dataset quite a lot

## Modify time
Convert the timestamp into a numeric value and then into a string so that the time of day and month can be categorised.

In [10]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)

+-----+--------+-------------------+---------+----------+------------+------+------+----+------+----------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+
|  TMC|Severity|         Start_Time|Start_Lat| Start_Lng|Distance(mi)|Number|Street|Side|  City|    County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_S

## Quantiles function
Alternative solution to get position 
Remove City, Country, State, Zipcode, Airport_Code
In order to reduce dimensionality feature crossing will be applied upon the Start_Lat and Start_Lng then fused in order to create a new set of data

In [11]:
# TODO - Calculate how many buckets needed (Bins)
discretizer_Lat = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lat", outputCol="Start_Lat_disc")
discretizer_Lng = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lng", outputCol="Start_Lng_disc")

# Into categorical values
indexer_cord = [StringIndexer(inputCol=c + "_disc", outputCol=c+"_IDX") for c in ["Start_Lat","Start_Lng"]]

# One-hot (feature crossing)
encoder_cord = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexer_cord], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexer_cord])

pipeline_cord = Pipeline(stages=[discretizer_Lat,discretizer_Lng, *indexer_cord, encoder_cord])
preprocessed_cord = pipeline_cord.fit(df).transform(df)

position = VectorAssembler(inputCols=["Start_Lat_IDX_vec","Start_Lng_IDX_vec"], outputCol="position")

df = position.transform(preprocessed_cord)

In [12]:
df.show(1)

+-----+--------+-------------------+---------+----------+------------+------+------+----+------+----------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+--------------+--------------+-------------+-------------+-----------------+-----------------+--------------------+
|  TMC|Severity|         Start_Time|Start_Lat| Start_Lng|Distance(mi)|Number|Street|Side|  City|    County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossi

## Cast data 
Cast all the datacolumns into correct format so they will be sorted to numerical or categorical values. Data removed caused to big feature vector, contained too many NaN values or were converted into another form.

In [13]:
df = df.select(
        #col('TMC').cast('string'),  Lot of missing values!!
        col('Severity').cast('int'),
        #col('Start_Time').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        col('position'), # already preprocessed
        col('Distance(mi)').cast('double'),
        col('Side').cast('string'),         # Remove eventually?
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'), #  Lot of missing values!!
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'),  #Lot of missing values!!
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

# Since the last check can be unorganised we recreate a new list that contains all data
#df.cache()  # TODO does this work? - Heatbeat crash otherwise
colLabel = ["Severity"]

rdd = sc.parallelize(df.dtypes)
colCat = rdd.map(lambda i: i[0] if (i[1]=='string' or i[1]=='boolean' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()
colNum = rdd.map(lambda i: i[0] if (i[1]=='double' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()

In [14]:
print(colLabel)
print(colCat)
print(colNum)

['Severity']
['Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'Side', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
['Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']


## Clean data

### Recheck the missing values
Check so that the output contains 0 missing values

In [15]:
df_missingVals_cols = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
# Missing value in each column
df_missingVals_cols.collect()

[Row(Severity=0, Start_Hour=3163, Start_Month=3163, Weather_Hour=36705, Weather_Month=36705, position=0, Distance(mi)=0, Side=0, Temperature(F)=56063, Wind_Chill(F)=1852623, Humidity(%)=59173, Pressure(in)=48142, Visibility(mi)=65691, Wind_Direction=45101, Wind_Speed(mph)=440840, Weather_Condition=65932, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Sunrise_Sunset=93, Civil_Twilight=93, Nautical_Twilight=93, Astronomical_Twilight=93)]

### Drop NaN
Cant drop NaN since imputer should take care of the most of them! However we can drop values from the categorical set if we want!

In [16]:
#old = df.count()
#df = df.na.drop()
#print("Rows removed:",old-df.count())

### Recheck the missing values
Check so that the output contains 0 missing values

In [ ]:
df_missingVals_cols = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
# Missing value in each column
df_missingVals_cols.collect()

### Remove data with occurance less than 1%
Based on information from analysis. With further analysis lower procentage can be used to find better results.

In [ ]:
original_rows = df.count()
weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)

n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit
df_filtered = weather_freq.filter(weather_freq['count'] > n)

filtered_conditions = df_filtered.select("Weather_Condition").rdd.flatMap(lambda x: x).collect()
df = df.filter(df['Weather_Condition'].isin(*filtered_conditions))

df_filtered.show()
print(filtered_conditions)
print("Rows removed:",original_rows - df.count())

## Prepare Pipeline

### Impute 

In [ ]:
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

### Scaling

In [ ]:
num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features",handleInvalid="keep")
#scaler = StandardScaler(inputCol="num_features", withMean=True, withStd=True, outputCol="scaledFeatures")
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol="num_features", outputCol="scaledFeatures")

### Indexer

In [ ]:
indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='keep') for c in colCat]

### Encoder

In [ ]:
encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexers],handleInvalid="keep")

## Preprocessing - Pipeline

In [ ]:
# Categorical values

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

pipeline = Pipeline(stages=[numPipeline, catPipeline])
preprocessed_df = pipeline.fit(df).transform(df)

In [ ]:
preprocessed_df.select("position","scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(1,False)

## Combine features

In [ ]:
va2 = VectorAssembler(inputCols=["position","scaledFeatures",*[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

df = va2.transform(preprocessed_df)

df = df.withColumn('label', col("Severity"))
df_features = df.withColumn('features', df.final_features).select("features","label")
df_features.show(1, False)
print("Feature set size:",df_features.count())

## Count categorical values

In [ ]:
# Checking how many classes that can be used
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat, "position"]] 
print("Unique column values:", tmp)

# 3. Feature importance

In [ ]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.feature import PCA

## ChiSqSelector
Check top 50 which of the values in the feature vector

In [ ]:
selector = ChiSqSelector(numTopFeatures=50, 
                         labelCol='label', 
                         featuresCol='features', 
                         outputCol="selectedFeatures",
                         selectorType='numTopFeatures', 
                         percentile=0.1, 
                         fpr=0.05, fdr=0.05, fwe=0.05)

model = selector.fit(df_features)
chi_model = model.transform(df_features)
chi_model.head().selectedFeatures

print("Top selected features according to ChiSqSelector:", model.selectedFeatures)

## PCA
In order to understand how much the variance affect the dataset we check with PCA

In [ ]:
pca = PCA(k=50, inputCol="features", outputCol="pca_features")
model = pca.fit(df_features)
chi_model = model.transform(df_features)

chi_model[0].pca_features

model.explainedVariance


# 4. Machine learning

In [ ]:
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier,GBTClassifier,MultilayerPerceptronClassifier
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [ ]:
trainSet, testSet = df_features.randomSplit([0.8 ,0.2], 1)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [ ]:
trainSet.show(2,False)

## Logistic regression

In [ ]:
lr = LogisticRegression(labelCol="label", featuresCol="features")

paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1]) \
                .addGrid(lr.maxIter, [10]) \
                .addGrid(lr.elasticNetParam, [0.6]) \
                .build()
                #.addGrid(lr.regParam, [0.1, 0.01,0.02]) \
                #.addGrid(lr.maxIter, [5,10,15]) \
                #.addGrid(lr.elasticNetParam, [0.6,0.8]) \
cv =  CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10, seed=None, parallelism=8) 
cvModel_lr = cv.fit(trainSet)

predictions = cvModel_lr.transform(testSet)
predictions.select("prediction", "label", "features").show(5)
accuracy = evaluator.evaluate(predictions)

trainSummary = cvModel_lr.bestModel.summary
# TODO: What was the best parameters?
print("Accuracy:",accuracy)

In [ ]:
cvModel_lr.getEstimatorParamMaps()[ argmax(cvModel.avgMetrics) ]

In [ ]:
cvModel_lr.save("/Models/LR_Model")

## Decision tree

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features") 

paramGrid = ParamGridBuilder() .build()

cv =  CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10, seed=None, parallelism=8) 
cvModel_dt = cv.fit(trainSet)

predictions = cvModel_dt.transform(testSet)
predictions.select("prediction", "label", "features").show(5)
accuracy = evaluator.evaluate(predictions)
print("Accuracy:",accuracy)

In [ ]:
cvModel_dt.getEstimatorParamMaps()[ argmax(cvModel.avgMetrics) ]

## Random forest

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features") #numTrees=10
paramGrid = ParamGridBuilder() \
                .addGrid(rf.numTrees,[10,15,20,25]) \
                .build()

cv =  CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10, seed=None, parallelism=8) 
cvModel_rf = cv.fit(trainSet)

predictions = cvModel_rf.transform(testSet)
predictions.select("prediction", "label", "features").show(5)
accuracy = evaluator.evaluate(predictions)
print("Accuracy:",accuracy)

In [ ]:
cvModel_rf.getEstimatorParamMaps()[ argmax(cvModel.avgMetrics) ]

## MLP (Not working)

In [ ]:
#mlp = MultilayerPerceptronClassifier(maxIter=100, blockSize=128, layers=[4, 5, 4, 3], seed=1234) #Layers=[]
#paramGrid = ParamGridBuilder().build()
                #.addGrid(gbt.maxIter, [10,15,20]) \
                

#cv =  CrossValidator(estimator=mlp, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10, seed=42) 
#cvModel = cv.fit(trainSet)

#predictions = cvModel.transform(testSet)
#predictions.select("prediction", "label", "features").show(5)
#accuracy = evaluator.evaluate(predictions)
#print("Accuracy:",accuracy)

## Scikit Evaluation

## Hyperparameter tuning of best model

Since there is no predefined way to test multiple algorithms this is used!

In [ ]:
#models = createParams(models)
#bestModels = evaluateAlgorithm(models,trainSet)
#finalModel = getBestModel(bestModels,testSet)

#prediction = makePredictions(finalModel, testSet)
trainingSummary = finalModel.summary

# Make predictions on test documents. cvModel uses the best model found (lrModel).


#trainingSummary = cvModel.bestModel.stages[-1].summary
#selected = prediction.select("label", "probability", "prediction")
#selected.show(10)

In [ ]:
#finalModel = getBestModel(bestModels,testSet)
#print(finalModel)

In [ ]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
falsePositiveRateLabel = trainingSummary.falsePositiveRateByLabel
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
iterations = trainingSummary.totalIterations

print("Iterations: %s\nAccuracy: %s\nFPR: %s\nTPR: %s\nFPRL: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (iterations,accuracy, falsePositiveRate, truePositiveRate, falsePositiveRateLabel, fMeasure, precision, recall))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = prediction.select(['label']).collect()
y_pred = prediction.select(['prediction']).collect()

print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

# TODO try streaming (online-learning)